In [11]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from dateutil.relativedelta import relativedelta
import math
import re
import pyodbc



# Reading Source Data 'ADT_Dummy Data' and 'ADT_ER_IP_04012021to07062021'

server = 'wn000038725' 
database = 'oas_mas_stg' 
username = 'svcoimas'
password = 'Canopyanalytics#124'

#cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()


df = pd.read_excel(r'C:\Users\ssing296\Desktop\Canopy\MyCanopy Dashboard\optum_registration_report_4_2.xls')

#df['Diagnosis'].fillna('blank_code', inplace=True)

qry_insert = "Insert into  [Canopy].[Registration_Master_1] ([Date of Registration],[Member ID of Registrant],[Kinvey ID of Registrant],[Member ID of Primary Subscriber],[Kinvey ID of subscriber],[Group ID of registrant],[Payor of Registrant],[IPA of Registrant],[Category Name of Registrant],[DOB of Registrant],[Age]) values(?,?,?,?,?,?,?,?,?,?,?)"

#cursor.execute("truncate table [Canopy].[ARCH_ADT_ANDOR_IP_input]")

def check_null(column_value):
    if(pd.isnull(column_value)):
        column_value = None
    else:
        column_value
    return column_value

insert_batch = []
rec = ""
counter = 0;
batchnum = 1;

data_len = len(df)


for index,row in df.iterrows():

    rec = (check_null(row['Date of Registration']),check_null(row[' Member ID of Registrant']),row[' Kinvey ID of Registrant'],row[' Member ID of Primary Subscriber'],row[' Kinvey ID of subscriber'],row[' Group ID of registrant'],check_null(row[' Payor of Registrant']),check_null(row[' IPA of Registrant']),row[' Category Name of Registrant'],check_null(row[' DOB of Registrant']),check_null(row['Age'])) 
    insert_batch.append(rec)
    counter = counter + 1;
    if(counter%100 == 0):
        cursor.executemany(qry_insert,insert_batch)
        cursor.commit()
        counter = 0
        data_len = data_len - len(insert_batch)
        insert_batch = []
        batchnum = batchnum + 1

if data_len != 0:
    cursor.executemany(qry_insert,insert_batch)  

    cursor.execute(qry_insert,rec)
    
print("Data Load Done")

cnxn.commit()
cursor.close()

Data Load Done
